In [1]:
import os
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
plt.rcParams["figure.dpi"] = 150

from NuRadioReco.utilities import units
from matplotlib.backends.backend_pdf import PdfPages

# Notebook to generate Veffs from single json files per output directory

Adjust the following input paths if need be

In [2]:
files = glob("veffs_with_combined/*.json") # individual json files

review_files =  {"e": '../review_array_dict_e.json', "mu": '../review_array_dict_mu.json', "tau": '../review_array_dict_tau.json'}

plot_review = True
for key in review_files:
    if not os.path.exists(review_files[key]):
        print(f"{review_files[key]} does not exist, will not plot review veff for comparison")
        plot_review = False
        
print(f"Found input Veff files: {len(files)}")


Found input Veff files: 620


In [3]:
# read them all in and convert to pandas table
dataframes = []
for f in files:
    df = pd.read_json(f)
    a, df["detector"], df["config_file"], df["sim_file"], df["flavor"], df["filename"] = f.split("__")
    dataframes.append(df)
data = pd.concat(dataframes)
data.sort_values(["detector", "flavor", "energy","thetamin"], inplace=True)
data.reset_index(inplace=True)

In [4]:
def get_Veff_Vefferr_for_trigger(df, name='LPDA_2of4_100Hz'):
    V = []
    err = []
    for i, entry in df.veff.iteritems():
        #print(i,entry)
        V.append(entry[name][0])
        err.append(entry[name][1])
    return np.array(V), np.array(err)


triggers = list(df.veff[0].keys())
flavors = list(set(list(data.flavor)))
detectors = list(set(list(data.detector)))

print(f"TRIGGERS {triggers}")
print(f"FLAVORS {flavors}")
print(f"TDETECTORS {detectors}")

for trigger in triggers:
        data[trigger], data["err_"+trigger] = get_Veff_Vefferr_for_trigger(data, name=trigger)

TRIGGERS ['LPDA_2of4_100Hz', 'LPDA_2of4_10mHz', 'PA_4channel_100Hz', 'PA_4channel_1mHz', 'PA_8channel_100Hz', 'PA_8channel_1mHz', 'all_triggers', 'combined_4channelPA', 'combined_4channelPA_highThreshold', 'combined_8channelPA', 'combined_8channelPA_highThreshold']
FLAVORS ['e', 'mu', 'tau']
TDETECTORS ['baseline_array', 'hex_hybrid_only_array', 'hex_shallowheavy_array', 'hex_shallow_array']


## Some stats on files per directory / triggered events / generated events

In [6]:
%%capture out
figi = 0
with PdfPages('TDRprod_number_triggered_any.pdf') as pdf:
    for flav in flavors:
        for det in detectors:
            figi += 1
            sel_data = data[(data.flavor==flav) & (data.detector==det)]
            energies = list(set(list(sel_data.energy)))
            energies.sort()
            #plt.clf()
            plt.figure(figi)
            for e in energies:
                sd = sel_data[sel_data.energy==e]
                plt.plot(np.cos(sd.thetamin), sd.stats_ntrig, "o-", label=f"E={e:.1e}eV")
            plt.semilogy()
            plt.legend()
            plt.title(f"flavor: {flav}, detector: {det}")
            plt.ylim(1,None)
            plt.xlim(-1.05,1.05)
            plt.xlabel("cos(zenith)")
            plt.ylabel("number of triggered events")
            pdf.savefig()

In [7]:
%%capture out
figi = 0
with PdfPages('TDRprod_number_events_simulated.pdf') as pdf:
    for flav in flavors:
        for det in detectors:
            figi += 1
            sel_data = data[(data.flavor==flav) & (data.detector==det)]
            energies = list(set(list(sel_data.energy)))
            energies.sort()
            #plt.clf()
            plt.figure(figi)
            for e in energies:
                sd = sel_data[sel_data.energy==e]
                plt.plot(np.cos(sd.thetamin), sd.stats_nsim, "o-", label=f"E={e:.1e}eV")
            plt.semilogy()
            plt.legend()
            plt.title(f"flavor: {flav}, detector: {det}")
            plt.ylim(1e4,1e7)
            plt.xlim(-1.05,1.05)
            plt.xlabel("cos(zenith)")
            plt.ylabel("number of triggered events")
            pdf.savefig()

In [12]:
%%capture out
figi = 0
with PdfPages('TDRprod_number_files_simulated.pdf') as pdf:
    for flav in flavors:
        for det in detectors:
            figi += 1
            sel_data = data[(data.flavor==flav) & (data.detector==det)]
            energies = list(set(list(sel_data.energy)))
            energies.sort()
            #plt.clf()
            plt.figure(figi)
            for e in energies:
                sd = sel_data[sel_data.energy==e]
                plt.plot(np.cos(sd.thetamin), sd.nfiles, "o-", label=f"E={e:.1e}eV")
            plt.semilogy()
            plt.legend()
            plt.title(f"flavor: {flav}, detector: {det}")
            plt.ylim(1,None)
            plt.xlim(-1.05,1.05)
            plt.xlabel("cos(zenith)")
            plt.ylabel("number of triggered events")
            pdf.savefig()

# Plot Veffs vs cos(zenith)

In [8]:
%%capture out
figi = 0
with PdfPages('TDRprod_Veff_vs_cosz.pdf') as pdf:
    for flav in flavors:
        for det in detectors:
            sel_data = data[(data.flavor==flav) & (data.detector==det)]
            energies = list(set(list(sel_data.energy)))
            energies.sort()
            for energy in energies:
                df = sel_data[sel_data.energy==energy]
                #print(flav, det)
                df.reset_index(inplace=True)

                figi +=1
                plt.figure(figi)

                ##plt.clf()

                #do plotting
                for t in triggers:
                    # do not plot combinations for now
                    if t.startswith("combined"):
                        continue
                    plt.plot(0.5*(np.cos(df.thetamax)+np.cos(df.thetamin)), df[t]/units.km**3, "o-", label=t)
                    plt.fill_between(0.5*(np.cos(df.thetamax)+np.cos(df.thetamin)), (df[t]+df["err_"+t])/units.km**3, (df[t]-df["err_"+t])/units.km**3, alpha=0.2)

                if plot_review:
                    review_filename = review_files[flav]
                    df_review = pd.read_json(review_filename)

                    df_review["combined_array"], df_review["err_combined_array"] = get_Veff_Vefferr_for_trigger(df_review, name="combined_array")
                    df_review = df_review[df_review["energy"]==energy]
                    plt.plot(0.5*(np.cos(df_review.thetamax)+np.cos(df_review.thetamin)), df_review["combined_array"]/units.km**3, ":", label="review array (d+s)")

                plt.title(f"flavor: {flav}, detector: {det}, E: {energy:.1e} eV")
                plt.legend(ncol=2)
                plt.xlim(-1,1)
                plt.semilogy()
                plt.ylim(0.01,1e4)#None)
                plt.xlabel("cos(zenith)")
                plt.ylabel(r"Veff [km$^{3}$]")
                pdf.savefig() 


                #plt.savefig(filename.replace(".json", ".pdf"))

In [9]:
%%capture out
figi = 0
with PdfPages('TDRprod_Veff_vs_cosz_combined.pdf') as pdf:
    for flav in flavors:
        for det in detectors:
            sel_data = data[(data.flavor==flav) & (data.detector==det)]
            energies = list(set(list(sel_data.energy)))
            energies.sort()
            for energy in energies:
                df = sel_data[sel_data.energy==energy]
                #print(flav, det)
                df.reset_index(inplace=True)

                figi +=1
                plt.figure(figi)

                ##plt.clf()

                #do plotting
                for t in triggers:
                    # do not plot combinations for now
                    if t not in ['LPDA_2of4_100Hz', 'PA_4channel_100Hz', 'PA_8channel_100Hz', 'combined_4channelPA', 'combined_8channelPA']:
                        continue
                    plt.plot(0.5*(np.cos(df.thetamax)+np.cos(df.thetamin)), df[t]/units.km**3, ".-", label=t)
                    plt.fill_between(0.5*(np.cos(df.thetamax)+np.cos(df.thetamin)), (df[t]+df["err_"+t])/units.km**3, (df[t]-df["err_"+t])/units.km**3, alpha=0.2)

                if plot_review:
                    review_filename = review_files[flav]
                    df_review = pd.read_json(review_filename)

                    df_review["combined_array"], df_review["err_combined_array"] = get_Veff_Vefferr_for_trigger(df_review, name="combined_array")
                    df_review = df_review[df_review["energy"]==energy]
                    plt.plot(0.5*(np.cos(df_review.thetamax)+np.cos(df_review.thetamin)), df_review["combined_array"]/units.km**3, ":", label="review array (d+s)")

                plt.title(f"flavor: {flav}, detector: {det}, E: {energy:.1e} eV")
                plt.legend(ncol=2)
                plt.xlim(-1,1)
                plt.semilogy()
                plt.ylim(0.01,1e4)#None)
                plt.xlabel("cos(zenith)")
                plt.ylabel(r"Veff [km$^{3}$]")
                pdf.savefig() 


                #plt.savefig(filename.replace(".json", ".pdf"))

In [10]:
%%capture out
figi = 0
with PdfPages('TDRprod_Veff_vs_E_per_array.pdf') as pdf:
    for flav in flavors:
        figi +=1
        plt.figure(figi)
        for det in detectors:
            sel_data = data[(data.flavor==flav) & (data.detector==det)]
            energies = list(set(list(sel_data.energy)))
            energies.sort()
                
            df = sel_data[sel_data.energy==energy]
            #print(flav, det)
            df.reset_index(inplace=True)

            ##plt.clf()

            #do plotting
            for t in triggers:
                # do not plot combinations for now
                if t == 'combined_4channelPA':#, 'combined_8channelPA']:
                    plt.plot(energies, np.array([np.sum(sel_data[sel_data.energy==energy][t])/20. for energy in energies])/units.km**3, ".-", label=det)
                    print(len(sel_data[sel_data.energy==energy]))
                #elif t  in ['PA_4channel_100Hz']:
                #    plt.plot(energies, np.array([np.sum(sel_data[sel_data.energy==energy][t])/20. for energy in energies])/units.km**3, ".:", label="PA only")
                #plt.fill_between(0.5*(np.cos(df.thetamax)+np.cos(df.thetamin)), (df[t]+df["err_"+t])/units.km**3, (df[t]-df["err_"+t])/units.km**3, alpha=0.2)
            
        if plot_review:
            review_filename = review_files[flav]
            df_review = pd.read_json(review_filename)

            df_review["combined_array"], df_review["err_combined_array"] = get_Veff_Vefferr_for_trigger(df_review, name="combined_array")
            review_energies = list(set(list(df_review.energy)))
            review_energies.sort()
            plt.plot(review_energies, np.array([np.sum(df_review[df_review.energy==energy]["combined_array"])/20. for energy in review_energies])/units.km**3, ":", label="review array (d+s)")
            print(len(df_review[df_review.energy==energy]))
                  
        plt.title(f"flavor: {flav}")
        plt.legend(ncol=1)
        plt.xlim(10**15.9, 10**18.6)
        plt.loglog()
        plt.ylim(0,400)#None)
        plt.xlabel("energy [GeV]")
        plt.ylabel(r"Veff [km$^{3}$]")
        pdf.savefig() 


            #plt.savefig(filename.replace(".json", ".pdf"))

In [11]:
%%capture out
figi = 0
with PdfPages('TDRprod_Veff_vs_E_combined.pdf') as pdf:
    for det in detectors:
        figi +=1
        plt.figure(figi)
        for flav in flavors:
            sel_data = data[(data.flavor==flav) & (data.detector==det)]
            energies = list(set(list(sel_data.energy)))
            energies.sort()
                
            df = sel_data[sel_data.energy==energy]
            #print(flav, det)
            df.reset_index(inplace=True)

            ##plt.clf()

            #do plotting
            for t in triggers:
                # do not plot combinations for now
                if t == 'combined_4channelPA':#, 'combined_8channelPA']:
                    plt.plot(energies, np.array([np.sum(sel_data[sel_data.energy==energy][t])/20. for energy in energies])/units.km**3, ".-", label=f"{det}, {flav}")
                    print(len(sel_data[sel_data.energy==energy]))
                #elif t  in ['PA_4channel_100Hz']:
                #    plt.plot(energies, np.array([np.sum(sel_data[sel_data.energy==energy][t])/20. for energy in energies])/units.km**3, ".:", label="PA only")
                #plt.fill_between(0.5*(np.cos(df.thetamax)+np.cos(df.thetamin)), (df[t]+df["err_"+t])/units.km**3, (df[t]-df["err_"+t])/units.km**3, alpha=0.2)
            
            if plot_review:
                review_filename = review_files[flav]
                df_review = pd.read_json(review_filename)

                df_review["combined_array"], df_review["err_combined_array"] = get_Veff_Vefferr_for_trigger(df_review, name="combined_array")
                review_energies = list(set(list(df_review.energy)))
                review_energies.sort()
                plt.plot(review_energies, np.array([np.sum(df_review[df_review.energy==energy]["combined_array"])/20. for energy in review_energies])/units.km**3, ":", label=f"review array, {flav}")
                print(len(df_review[df_review.energy==energy]))

        plt.legend(ncol=1)
        plt.xlim(10**15.9, 10**18.6)
        plt.loglog()
        plt.ylim(0,400)#None)
        plt.xlabel("energy [GeV]")
        plt.ylabel(r"Veff [km$^{3}$]")

            
            
        pdf.savefig() 


            #plt.savefig(filename.replace(".json", ".pdf"))